Computer vision

In [3]:
import cv2   #pip install opencv-contrib-python

In [2]:
video=cv2.VideoCapture('sample.mp4')

In [3]:
while True:
    state,frame=video.read()
    if not state:
        break
    cv2.imshow('video output',frame)
    if cv2.waitKey(1)==27:
        break
video.release()
cv2.destroyAllWindows()

In [8]:
def webcam(webcam_idx=0):
    video=cv2.VideoCapture(webcam_idx)
    while True:
        state,frame=video.read()
        if not state:
            break
        cv2.imshow('video output',frame)
        if cv2.waitKey(1)==27:
            break
video.release()
cv2.destroyAllWindows()

In [9]:
webcam()

In [1]:
def recorder(webcam_idx=0,recording_file='recording.avi'):
    video=cv2.VideoCapture(webcam_idx)
    fourcc=cv2.VideoWriter_fourcc(*'XVID')
    out=cv2.VideoWriter(recording_file,fourcc,20.0,(640,480))
    while True:
        state,frame=video.read()
        if not state:break
        cv2.imshow('video camera',frame)
        out.write(frame)
        if cv2.waitKey(1)==27:
            break
    out.release()
    video.release()
    cv2.destroyAllWindows()

In [6]:
recorder(recording_file='example_webcam.avi')

In [10]:
def webcam_w_info(webcam_idx=0):
    video=cv2.VideoCapture(webcam_idx)
    while True:
        state,frame=video.read()
        if not state:
            break
        cv2.rectangle(frame,(0,0),(500,20),(255,255,0),-1)
        cv2.putText(frame,"webcam output from laptop",(10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0))
        cv2.putText(frame,'camera o',(10,450),cv2.FONT_HERSHEY_SIMPLEX,.5,(0,255,255))
        cv2.imshow('video output',frame)
        if cv2.waitKey(1)==27:
            break
    video.release()
    cv2.destroyAllWindows()
webcam_w_info()


In [2]:
import mediapipe as mp

In [4]:
mp_drawing=mp.solutions.drawing_utils
mp_drawing_styles=mp.solutions.drawing_styles
mp_hands=mp.solutions.hands
IMAGE_FILES=[]
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
  for idx,file in enumerate(IMAGE_FILES):
    image=cv2.flip(cv2.imread(file),1)
    results=hands.process(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
    print('Handedness:',results.multi_handedness)
    if not results.multi_hand_landmarks:
        continue
    image_height,image_width,_=image.shape
    annotated_image=image.copy()
    for hand_landmarks in results.multi_hand_landmarks:

      print('hand_landmarks:', hand_landmarks)
      print(
          f'Index finger tip coordinates: (',
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
      )
      mp_drawing.draw_landmarks(
          annotated_image,
          hand_landmarks,
          mp_hands.HAND_CONNECTIONS,
          mp_drawing_styles.get_default_hand_landmarks_style(),
          mp_drawing_styles.get_default_hand_connections_style())
    cv2.imwrite(
        '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))
    # Draw hand world landmarks.
    if not results.multi_hand_world_landmarks:
      continue
    for hand_world_landmarks in results.multi_hand_world_landmarks:
      mp_drawing.plot_landmarks(
        hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()